In [ ]:
DECLARE  @rafo_model VARBINARY(MAX)
IF @rec_segment > 120   
    SET @rafo_model = (SELECT rafo_model FROM dbo.RAFO_IFC_DB_anchor_121models_3
    WHERE car_segment= 121 )
ELSE  
    SET @rafo_model = (SELECT rafo_model FROM dbo.RAFO_IFC_DB_anchor_121models_3 
    WHERE car_segment= @rec_segment );  

EXECUTE dbo.price_record_DB_calculation @rafo_model = @rafo_model,
                                        @predicted_price = @predicted_price OUTPUT,
                                        @predicted_min =  @predicted_min OUTPUT,
                                        @predicted_max =  @predicted_max OUTPUT,
                                        @predicted_median =  @predicted_median OUTPUT,
                                        @predicted_std =  @predicted_std OUTPUT

UPDATE coded_DB_anchor4_record
SET pred_price = @predicted_price
WHERE model=@rec_model;

INSERT INTO pricing_results
VALUES ( @predicted_price, @predicted_min, @predicted_max, @predicted_median,  @predicted_std  );

In [ ]:
USE [Pricing]
GO

DECLARE @return_value int,
        @predicted_price int,
        @predicted_min int,
        @predicted_max int,
        @predicted_median int,
        @predicted_std int,
        @rec_segment int,
        @anchor int,
        @age int,
        @make_model nvarchar(40),
        @revage float,
        @revkm float,
        @make_cd int,
        @model_cd int,
        @body_cd int,
        @fuel_cd int,
        @km_range int,
        @age_years int,
        @map_anchor int,
        @year_med_anchor int,
        @med_anchor int,
        @rec_model nvarchar(30)

EXEC    @return_value = [dbo].[complete_record_DB_price_interval_calculation]
@record_make = N'skoda',
@record_model = N'octavia',
@record_body = N'combi',
@record_fueltype = N'diesel',
@record_kilometer = 40000,
@record_makeyear = 2015,
@reg_month = 1,
@record_power = 77,
@record_sizeccm = 1568,
@record_automat = 0,
@record_drive = 0,
@mec_con = 3,
@ext_con = 3,
@record_isfirstowner = 0,
@record_isservicebook = 1,
@record_isvat = 0,
@record_autoaircondition = 0,
@record_airconditioned = 1,
@record_isxenon = 0,
@record_isled = 0,
@record_isnavigation = 0,
@record_iscruisecontrol = 0,
@record_isinleather = 0,
@record_airbags = 1,
@record_price = 0,
@predicted_price = @predicted_price OUTPUT,
@predicted_min = @predicted_min OUTPUT,
@predicted_max = @predicted_max OUTPUT,
@predicted_median = @predicted_median OUTPUT,
@predicted_std = @predicted_std OUTPUT,
@rec_segment = @rec_segment OUTPUT,
@anchor = @anchor OUTPUT,
@age = @age OUTPUT,
@make_model = @make_model OUTPUT,
@revage = @revage OUTPUT,
@revkm = @revkm OUTPUT,
@make_cd = @make_cd OUTPUT,
@model_cd = @model_cd OUTPUT,
@body_cd = @body_cd OUTPUT,
@fuel_cd = @fuel_cd OUTPUT,
@km_range = @km_range OUTPUT,
@age_years = @age_years OUTPUT,
@map_anchor = @map_anchor OUTPUT,
@year_med_anchor = @year_med_anchor OUTPUT,
@med_anchor = @med_anchor OUTPUT,
@rec_model = @rec_model OUTPUT

SELECT  @predicted_price as N'@predicted_price',
@predicted_min as N'@predicted_min',
@predicted_max as N'@predicted_max',
@predicted_median as N'@predicted_median',
@predicted_std as N'@predicted_std',
@rec_segment as N'@rec_segment',
@anchor as N'@anchor',
@age as N'@age',
@make_model as N'@make_model',
@revage as N'@revage',
@revkm as N'@revkm',
@make_cd as N'@make_cd',
@model_cd as N'@model_cd',
@body_cd as N'@body_cd',
@fuel_cd as N'@fuel_cd',
@km_range as N'@km_range',
@age_years as N'@age_years',
@map_anchor as N'@map_anchor',
@year_med_anchor as N'@year_med_anchor',
@med_anchor as N'@med_anchor',
@rec_model as N'@rec_model'

SELECT	'Return Value' = @return_value

GO


In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[complete_record_DB_price_interval_calculation]
        Script Date: 6/29/2018 10:42:08 AM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[complete_record_DB_price_interval_calculation](
      @record_make nvarchar(20),
      @record_model nvarchar(20),
      @record_body nvarchar(20),
      @record_fueltype nvarchar(10),
      @record_kilometer int,
      @record_makeyear int,
      @reg_month int, 
      @record_power int,
      @record_sizeccm int,
      @record_automat int,
      @record_drive int,
      @mec_con int,
      @ext_con int,  
      @record_isfirstowner tinyint,
      @record_isservicebook tinyint,
      @record_isvat tinyint,
      @record_autoaircondition tinyint,
      @record_airconditioned tinyint,
      @record_isxenon tinyint,
      @record_isled tinyint,
      @record_isnavigation tinyint,
      @record_iscruisecontrol tinyint,
      @record_isinleather tinyint, 
      @record_airbags tinyint,
      @record_price int,
      @predicted_price int OUTPUT,
      @predicted_min int OUTPUT,
      @predicted_max int OUTPUT,
      @predicted_median int OUTPUT,
      @predicted_std int OUTPUT,
      @rec_segment int OUTPUT,
      @anchor int OUTPUT,
      @age int OUTPUT,
      @make_model nvarchar(40) OUTPUT,
      @revage float OUTPUT,
      @revkm float OUTPUT,
      @make_cd int OUTPUT,
      @model_cd int OUTPUT,
      @body_cd int OUTPUT,
      @fuel_cd int OUTPUT,
      @km_range int OUTPUT,
      @age_years int OUTPUT,
      @map_anchor int OUTPUT,
      @year_med_anchor int OUTPUT,
      @med_anchor int OUTPUT,
      @rec_model nvarchar(30) OUTPUT )

AS
BEGIN

DROP TABLE IF EXISTS coded_DB_anchor4_record;
CREATE TABLE [dbo].[coded_DB_anchor4_record]([make] [nvarchar](20) not NULL,
                                             [model] [nvarchar](20) not NULL,
                                             [make_model] [nvarchar](40) not NULL,
                                             [kilometer] [int] not NULL,
                                             [makeyear] [int] not NULL,
                                             [power] [int] not NULL,
                                             [sizeccm] [int] not NULL,
                                             [body] [nvarchar](20) not NULL,
                                             [fueltype] [nvarchar](10) not NULL,
                                             [automat] [int] NULL,
                                             [drive] [int] NULL,
                                             [mec_con] [int]  NULL,
                                             [ext_con] [int] NULL,
                                             [isfirstowner] [tinyint] NULL,
                                             [isservicebook] [tinyint] NULL,
                                             [isvat] [tinyint] NULL,
                                             [autoaircondition] [tinyint] NULL,
                                             [airconditioned] [tinyint] NULL,
                                             [isxenon] [tinyint] NULL,
                                             [isled] [tinyint] NULL,
                                             [isnavigation] [tinyint] NULL,
                                             [iscruisecontrol] [tinyint] NULL,
                                             [isinleather] [tinyint] NULL, 
                                             [airbags] [tinyint] NULL,
                                             [age] [int] NULL,
                                             [revage] [float] NULL,
                                             [revkm] [float] NULL,
                                             [make_cd] [int] NULL,
                                             [model_cd] [int] NULL,
                                             [body_cd] [int] NULL,
                                             [fuel_cd] [int] NULL,
                                             [km_range] [int] NULL,
                                             [age_years] [int] NULL,
                                             [segment] [int] NULL,
                                             [map_anchor] [int] NULL, 
                                             [year_med_anchor] [int] NULL,
                                             [med_anchor] [int] NULL,
                                             [anchor] [int] NULL,
                                             [price] [int] NULL,
                                             [pred_price] [int] NULL  );

DROP TABLE IF EXISTS [dbo].[pricing_results];
CREATE TABLE [dbo].[pricing_results]([pred_price] [int] NULL,
                                     [pred_min] [int] NULL,
                                     [pred_max] [int] NULL,
                                     [pred_median] [int] NULL,
                                     [pred_std] [int] NULL   );

EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np
import datetime as dt 
import time as tm
import datetime
cur_month = int(str(datetime.datetime.now())[5:7])
makes = ["lincoln","mg","fisker","iveco","lexus","infiniti","porsche","daewoo","rover","hummer",
         "subaru","smart","opel","mini","renault","fiat","mercedes","honda","toyota","seat",
         "chevrolet","lancia","bmw","ford","audi","citroen","volvo","skoda","peugeot","nissan",
         "mitsubishi","mazda","kia","vw","land rover","jaguar","hyundai","jeep","saab","alfa romeo",
         "vaz-lada","suzuki","dacia","daihatsu","dodge","cadillac","chrysler","ssang yong","isuzu",
         "alpina","aston martin","maserati","gaz","bentley","trabant","moskvic","tatra","awz",
         "zastava"]
models = model_code["make_model"].unique()
bodies = ["platform","van","sedan/saloon","hatchback","combi","compact mpv","pickup",
          "cooler - freezer","mpv","suv","coupe","bus","off road","convertible","boxvan",
          "tipper","tow car"]
fuels = ["diesel","petrol","cng","hybrid","lpg","elektromotor"]
def to_four(dv):
    if(dv==1):
        return 4
    return 0
if ((reg_month>12)|(reg_month==0)):
    reg_month = 1
cars1 = pd.DataFrame()  
cars1.loc[0,"make"] = record_make  
cars1.loc[0,"model"] = record_model 
cars1.loc[0,"kilometer"] = int(record_kilometer)
cars1.loc[0,"makeyear"] = int(round(int(record_makeyear),0))
cars1.loc[0,"power"] = int(round(int(record_power),0))
cars1.loc[0,"sizeccm"] = int(record_sizeccm) 
cars1.loc[0,"body"] = record_body  
cars1.loc[0,"fueltype"] = record_fueltype  
cars1.loc[0,"automat"] = int(record_automat) 
cars1.loc[0,"drive"] = int(to_four(record_drive))
cars1.loc[0,"mec_con"] = int(mec_con)
cars1.loc[0,"ext_con"] = int(ext_con)
cars1.loc[0,"price"] = int(record_price)
cols_to_fill = ["make","model","body","fueltype"]
for j in range(0,4):
    if (cars1.loc[0,cols_to_fill[j]] ==""):
        cars1.loc[0,cols_to_fill[j]] = "unknown"
cars1["make_model"] = cars1["make"] + cars1["model"] 
cars1.loc[0,"kilometer"] = 1+ cars1.loc[0,"kilometer"]
cars1.loc[0,"age"] = int(2*((2018-cars1.loc[0,"makeyear"])*12+cur_month-int(reg_month))+1)
addins = ["isfirstowner","isservicebook","isvat","autoaircondition","airconditioned",
          "isxenon","isled","isnavigation","iscruisecontrol","isinleather","airbags"]
impr_features = [record_isfirstowner, record_isservicebook, record_isvat, record_autoaircondition,
                 record_airconditioned, record_isxenon, record_isled, record_isnavigation,
                 record_iscruisecontrol, record_isinleather, record_airbags]
ilen = len(impr_features)
for i in range(0,ilen):
    cars1.loc[0, addins[i]] = int(impr_features[i])
columns_to_keep=["make","model","make_model","kilometer","age","power","sizeccm","body","fueltype",
                 "automat","drive","mec_con","ext_con","isfirstowner","isservicebook","isvat",
                 "autoaircondition","airconditioned","isxenon","isled","isnavigation","iscruisecontrol",
                 "isinleather","airbags","makeyear","price"]
cars1 = cars1[columns_to_keep]
mage = 26728
mkm = 18859
cars1["revage"] = mage/cars1["age"]
cars1["revkm"] = mkm/(cars1["kilometer"]/1000)
cars1["revage"] = cars1["revage"].apply(lambda rr: float("%.2f" %rr))
cars1["revkm"] = cars1["revkm"].apply(lambda rr: float("%.2f" %rr))
columns_to_mask = ["make","make_model","body","fueltype"]
cols_to_mask_codes = ["make_cd","model_cd","body_cd","fuel_cd"]
masks = [makes, models, bodies, fuels]
lens = [len(makes),len(models), len(bodies), len(fuels)]
def mask_make(mm,dl,mask):
    for i in range(0, dl):
        if ((mm == mask[i])==True):
            return i+1
        if (mm == "unknown"):
            return 0
    return dl+1
for j in range(0,4):
    cars1[cols_to_mask_codes[j]] = cars1[columns_to_mask[j]].apply(mask_make,args=(lens[j],masks[j]))   

columns_to_keep = ["make","model","make_model","kilometer","age","power","sizeccm","body","fueltype",
                   "automat","drive","mec_con","ext_con","isfirstowner","isservicebook","isvat",
                   "autoaircondition","airconditioned","isxenon","isled","isnavigation","iscruisecontrol",
                   "isinleather","airbags","makeyear","revage","revkm","make_cd","model_cd","body_cd",
                   "fuel_cd","price"]
cars1 = cars1[columns_to_keep] 
cars1["price"] = cars1["price"].apply(lambda rr: float("%.3f" %(int(rr/1000))))
cars1["segment"] = 0
age = int(cars1.loc[0,"age"])
revage = float(cars1.loc[0,"revage"])
revkm = float(cars1.loc[0,"revkm"]) 
make_cd = int(cars1.loc[0,"make_cd"]) 
model_cd = int(cars1.loc[0,"model_cd"]) 
body_cd = int(cars1.loc[0,"body_cd"])
fuel_cd = int(cars1.loc[0,"fuel_cd"])
columns_to_keep = ["make","model","make_model","kilometer","age","power","sizeccm","body","fueltype",
                   "automat","drive","mec_con","ext_con","isfirstowner","isservicebook","isvat",
                   "autoaircondition","airconditioned","isxenon","isled","isnavigation","iscruisecontrol",
                   "isinleather","airbags","makeyear","revage","revkm","make_cd","model_cd","body_cd",
                   "fuel_cd","price"]
cars1 = cars1[columns_to_keep] 
cars1["price"] = cars1["price"].apply(lambda rr: int(rr))
def fix_models(mm):
    if (mm=="fordfocus c-max"):
        return "fordfocus"
    if (mm=="renaultmegane scenic"):
        return "renaultscenic"
    if (mm=="opelzafira tourer"):
        return "opelzafira"
    if (mm=="seataltea xl"):
        return "seataltea"
    if (mm=="mazda323"):
        return "mazda3"
    if (mm=="nissanalmera tino"):
        return "nissanalmera"
    if (mm=="mercedesgle coupe"): 
        return "mercedesgle"                  
    if (mm=="kiapro_ceed"):
        return "kiaceed"
    if (mm=="hyundaiatos prime"):
        return "hyundaiatos"
    if (mm=="hyundaih-1 tour"):
        return "hyundaih-1"
    if (mm=="chryslerp. t. cruiser"):
        return "chryslerp. t. cruiser"
    if (mm=="toyotaland cruiser v8"):
        return "toyotaland cruiser"
    if (mm=="audiallroad a4"):
        return "audiallroad"
    return mm
cars1["make_model"] = cars1["make_model"].apply(fix_models)
def find_range(myp):
    if (myp<=10):
        return 0
    if ((myp>10)&(myp<=30)):
        return 1
    if ((myp>30)&(myp<=50)):
        return 2
    if ((myp>50)&(myp<=70)):
        return 3
    if ((myp>70)&(myp<=90)):
        return 4
    if ((myp>90)&(myp<=110)):
        return 5
    if ((myp>110)&(myp<=130)):
        return 6
    if ((myp>130)&(myp<=150)):
        return 7
    if ((myp>150)&(myp<=170)):
        return 8
    if ((myp>170)&(myp<=190)):
        return 9
    if ((myp>190)&(myp<=220)):
        return 10
    if ((myp>220)&(myp<=250)):
        return 11
    if ((myp>250)&(myp<=300)):
        return 12
    return 13
cars1.loc[0,"kilometer"] = float("%.3f" %(cars1.loc[0,"kilometer"]/1000))
cars1.loc[0,"km_range"] = find_range(cars1.loc[0,"kilometer"])
rec_model = record_model
make_model = cars1.loc[0,"make_model"]
km_range = int(cars1.loc[0,"km_range"])
def from2000_to_2017(yy):
    if(yy<2000):
        return 2000
    if(yy>2017):
        return 2017
    return yy
cars1.loc[0,"makeyear"] = from2000_to_2017(cars1.loc[0,"makeyear"])
cars1.loc[0,"age_years"] = int(2018 - cars1.loc[0,"makeyear"])
age_years = int(cars1.loc[0,"age_years"])
',
@input_data_1 = N'
SELECT * FROM dbo.model_codes ; 
',
@input_data_1_name = N'model_code',

@params = N'@record_make nvarchar(20), @record_model nvarchar(20), @record_body nvarchar(20),
            @record_fueltype nvarchar(10), @record_kilometer int, @record_makeyear int, @reg_month int, 
            @record_power int, @record_sizeccm int, @record_automat int, @record_drive int,
            @mec_con int, @ext_con int, @record_isfirstowner tinyint, @record_isservicebook tinyint,
            @record_isvat tinyint, @record_autoaircondition tinyint, @record_airconditioned tinyint,
            @record_isxenon tinyint, @record_isled tinyint, @record_isnavigation tinyint,
            @record_iscruisecontrol tinyint, @record_isinleather tinyint, @record_airbags tinyint,
            @record_price int, @age int OUTPUT, @make_model varchar(40) OUTPUT, @revage float OUTPUT,
            @revkm float OUTPUT, @make_cd int OUTPUT, @model_cd int OUTPUT,	@body_cd int OUTPUT,
            @fuel_cd int OUTPUT, @km_range int OUTPUT, @age_years int OUTPUT, @anchor int OUTPUT,
            @predicted_price int OUTPUT, @predicted_min int OUTPUT, @predicted_max int OUTPUT,
            @predicted_median int OUTPUT, @predicted_std int OUTPUT, @rec_segment int OUTPUT,
            @map_anchor int OUTPUT, @year_med_anchor int OUTPUT, @med_anchor int OUTPUT, 
            @rec_model nvarchar(30) OUTPUT
           ',
@record_make = @record_make, 
@record_model = @record_model, 
@record_body = @record_body,  
@record_fueltype = @record_fueltype,  
@record_kilometer = @record_kilometer,  
@record_makeyear = @record_makeyear,
@reg_month = @reg_month ,   
@record_power = @record_power,
@record_sizeccm = @record_sizeccm, 
@record_automat = @record_automat, 
@record_drive = @record_drive,
@mec_con = @mec_con,
@ext_con = @ext_con,
@record_isfirstowner = @record_isfirstowner,
@record_isservicebook = @record_isservicebook,
@record_isvat = @record_isvat,
@record_autoaircondition = @record_autoaircondition,
@record_airconditioned = @record_airconditioned,
@record_isxenon = @record_isxenon,
@record_isled = @record_isled, 
@record_isnavigation = @record_isnavigation,
@record_iscruisecontrol = @record_iscruisecontrol,
@record_isinleather = @record_isinleather,
@record_airbags = @record_airbags,
@record_price = @record_price,
@age = @age OUTPUT,
@make_model = @make_model OUTPUT,
@revage = @revage OUTPUT,
@revkm = @revkm OUTPUT, 
@make_cd = @make_cd OUTPUT, 
@model_cd = @model_cd OUTPUT, 
@body_cd = @body_cd OUTPUT,
@fuel_cd = @fuel_cd OUTPUT,
@km_range = @km_range OUTPUT,
@age_years = @age_years OUTPUT,
@anchor = @anchor OUTPUT,
@map_anchor = @map_anchor OUTPUT,
@year_med_anchor = @year_med_anchor OUTPUT,
@med_anchor = @med_anchor OUTPUT,
@predicted_price = @predicted_price OUTPUT,
@predicted_min = @predicted_min OUTPUT,
@predicted_max = @predicted_max OUTPUT,
@predicted_median = @predicted_median OUTPUT,
@predicted_std = @predicted_std OUTPUT,
@rec_segment = @rec_segment OUTPUT,
@rec_model = @rec_model OUTPUT
;

INSERT INTO coded_DB_anchor4_record
VALUES ( @record_make, @record_model,@make_model, @record_kilometer, @record_makeyear, @record_power,
         @record_sizeccm, @record_body, @record_fueltype, @record_automat, @record_drive,
         @mec_con, @ext_con, @record_isfirstowner, @record_isservicebook, @record_isvat, 
         @record_autoaircondition, @record_airconditioned, @record_isxenon, @record_isled, 
         @record_isnavigation, @record_iscruisecontrol, @record_isinleather, @record_airbags, 
         @age, @revage, @revkm, @make_cd, @model_cd, @body_cd, @fuel_cd, @km_range, @age_years,
         @rec_segment, 0,0,0,0, @record_price, 0);

SET @rec_segment = (SELECT anch.tscode
FROM dbo.TSorder_DB_codes anch INNER JOIN coded_DB_anchor4_record cotb
ON cotb.make_model=anch.make_model )
;
UPDATE coded_DB_anchor4_record
SET segment = @rec_segment 
WHERE model=@rec_model;

SET @anchor = (SELECT anch.anchor 
FROM dbo.Sauto_all20178_main_anchors anch INNER JOIN coded_DB_anchor4_record cotb
ON cotb.make_model=anch.make_model AND cotb.age_years=anch.age_years AND cotb.km_range=anch.km_range 
    AND cotb.[power]=anch.[power] )
;
IF (ISNUMERIC(@anchor) = 1)
    UPDATE coded_DB_anchor4_record
    SET anchor = @anchor  WHERE model=@rec_model;

SET @map_anchor = (SELECT anch.anchor_map 
FROM dbo.Sauto_all20178_map_nokm_anchors anch INNER JOIN coded_DB_anchor4_record cotb
ON cotb.make_model=anch.make_model AND cotb.age_years=anch.age_years AND cotb.[power]=anch.[power]);
IF (ISNUMERIC(@map_anchor) = 1)
    UPDATE coded_DB_anchor4_record
    SET map_anchor = @map_anchor WHERE model=@rec_model;
 
SET @year_med_anchor = (SELECT anch.anchor 
FROM dbo.Sauto_all2017_DB_mm_years_anchors anch INNER JOIN coded_DB_anchor4_record cotb
ON cotb.make_model=anch.make_model AND cotb.makeyear=anch.make_year );
IF (ISNUMERIC(@year_med_anchor) = 1)
    UPDATE coded_DB_anchor4_record
    SET year_med_anchor = @year_med_anchor WHERE model=@rec_model;

SET @med_anchor = (SELECT anch.anchor 
FROM dbo.Sauto_all2017_DB_make_model_anchors anch INNER JOIN coded_DB_anchor4_record cotb
ON cotb.make_model=anch.make_model );
IF (ISNUMERIC(@med_anchor) = 1)
    UPDATE coded_DB_anchor4_record
    SET med_anchor = @med_anchor WHERE model=@rec_model;

DECLARE  @rafo_model VARBINARY(MAX)
IF @rec_segment > 120   
    SET @rafo_model = (SELECT rafo_model FROM dbo.RAFO_IFC_DB_anchor_121models_3
    WHERE car_segment= 121 )
ELSE  
    SET @rafo_model = (SELECT rafo_model FROM dbo.RAFO_IFC_DB_anchor_121models_3 
    WHERE car_segment= @rec_segment );  

EXECUTE dbo.price_record_DB_calculation @rafo_model = @rafo_model,
                                        @predicted_price = @predicted_price OUTPUT,
                                        @predicted_min =  @predicted_min OUTPUT,
                                        @predicted_max =  @predicted_max OUTPUT,
                                        @predicted_median =  @predicted_median OUTPUT,
                                        @predicted_std =  @predicted_std OUTPUT

UPDATE coded_DB_anchor4_record
SET pred_price = @predicted_price
WHERE model=@rec_model;

INSERT INTO pricing_results
VALUES ( @predicted_price, @predicted_min, @predicted_max, @predicted_median,  @predicted_std  );

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[price_record_DB_calculation]    
        Script Date: 6/29/2018 3:46:16 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[price_record_DB_calculation]( 
                                               @rafo_model VARBINARY(MAX),
                                               @anchor int OUTPUT,
                                               @predicted_price int OUTPUT,
                                               @predicted_min int OUTPUT,
                                               @predicted_max int OUTPUT,
                                               @predicted_median int OUTPUT,
                                               @predicted_std int OUTPUT
                                                )
AS
BEGIN
  EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
    
import pandas as pd
import numpy as np
import pickle
from ipykernel import kernelapp as app
from sklearn.ensemble import RandomForestRegressor

if ((cars1.loc[0,"anchor"]==0)&(cars1.loc[0,"map_anchor"]>0)):
    cars1.loc[0,"anchor"] = cars1.loc[0,"map_anchor"]
if ((cars1.loc[0,"anchor"]==0)&(cars1.loc[0,"map_anchor"]==0)&(cars1.loc[0,"year_med_anchor"]>0)):
    cars1.loc[0,"anchor"] = cars1.loc[0,"year_med_anchor"]
if ((cars1.loc[0,"anchor"]==0)&(cars1.loc[0,"map_anchor"]==0)&(cars1.loc[0,"med_anchor"]>0)):
    cars1.loc[0,"anchor"] = cars1.loc[0,"med_anchor"]

pred_varTop = ["body_cd","fuel_cd","revkm","revage","power","sizeccm","automat","drive","mec_con",
               "ext_con","isfirstowner","isservicebook","isvat","autoaircondition","airconditioned",
               "isxenon","isled","isnavigation","iscruisecontrol","isinleather","airbags","anchor"]
pred_var = ["make_cd","model_cd","body_cd","fuel_cd","revkm","revage","power","sizeccm","automat",
            "drive","mec_con","ext_con","isfirstowner","isservicebook","isvat","autoaircondition",
            "airconditioned","isxenon","isled","isnavigation","iscruisecontrol","isinleather",
            "airbags","anchor"]
rafo = pickle.loads(rafo_model)
if (cars1.loc[0,"segment"] < 121):
    test_X = cars1[pred_varTop]
else:
    test_X = cars1[pred_var] 
test_y = cars1.price
nt = 33
tree_vals = [0]*nt
i = 0
for tree in range(nt):
    tree_vals[i] = int(rafo.estimators_[tree].predict(test_X)[0])
    i+=1
tree_vals_ary = np.array(tree_vals)
tree_vals_ary = np.sort(tree_vals_ary)
tree_vals_ary = np.delete(tree_vals_ary, np.s_[:int(nt/4)])
tree_vals_ary = np.delete(tree_vals_ary, np.s_[int(1+nt/2):])
cars1.loc[0,"pred_price"] = int(tree_vals_ary.mean())
predicted_price = int(cars1.loc[0,"pred_price"])
predicted_min = int(tree_vals_ary.min())
predicted_max = int(tree_vals_ary.max())
predicted_median = int(np.median(tree_vals_ary))
predicted_std = int(tree_vals_ary.std()) 
cars1.loc[0,"price"] = int(cars1.loc[0,"price"])
cars1.loc[0,"anchor"] = int(cars1.loc[0,"anchor"])
cars1.loc[0,"kilometer"] = int(cars1.loc[0,"kilometer"] -1)
if (cars1.loc[0,"anchor"] ==0):
    predicted_price = 0
    predicted_min = 0
    predicted_max = 0
anchor = int(cars1.loc[0,"anchor"])
OutputDataSet = cars1
',

@input_data_1 = N'
SELECT * FROM dbo.coded_DB_anchor4_record
',
@input_data_1_name = N'cars1',
@output_data_1_name = N'cars',
@params = N' @rafo_model varbinary(max) OUTPUT, @anchor int OUTPUT,
             @predicted_price int OUTPUT, @predicted_min int OUTPUT, @predicted_max int OUTPUT,
             @predicted_median int OUTPUT, @predicted_std int OUTPUT ',
@rafo_model = @rafo_model,
@anchor = @anchor OUTPUT,
@predicted_price = @predicted_price OUTPUT,
@predicted_min =  @predicted_min OUTPUT,
@predicted_max =  @predicted_max OUTPUT,
@predicted_median =  @predicted_median OUTPUT,
@predicted_std =  @predicted_std OUTPUT
;

END


In [ ]:
ALTER PROCEDURE [dbo].[all_on_click] 
AS
BEGIN
EXEC dbo.Sauto_preprocessing78 ;
EXEC dbo.Sauto_brand_mmcode_calculating78 ;
EXEC dbo.Sauto_anchors_calculating78 ;
EXEC dbo.Sauto_map_anchors_calculating78 ;
EXEC dbo.Sauto_mm_years_anchors_calculating78 ;
EXEC dbo.Sauto_make_model_anchors_calculating78 ;
EXEC dbo.sold20168_DB_TS_cars_processing ;              # calls dbo.sold20168_TS_cars_MMcoding
EXEC dbo.sold20168_DB_TS_cars_anchoring ;               # calls dbo.sold20168_TS_cars_ts_ordering
EXEC dbo.calculations_121_limit68_DB_RAFO_models ;
EXEC dbo.bought2018_Test_cars_processing46 ;
EXEC dbo.bought2018_Test_cars_anchoring ;

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_preprocessing78]  
        Script Date: 6/29/2018 3:32:02 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_preprocessing78] 
AS
BEGIN
DROP TABLE IF EXISTS dbo.Sauto_for_anchors_allMM;
CREATE TABLE dbo.Sauto_for_anchors_allMM ([make] [nvarchar](20) NULL,
                                          [model] [nvarchar](20) NULL,
                                          [make_model] [nvarchar](40) NULL,
                                          [makeyear] [int] NULL,
                                          [kilometer] [int] NULL,
                                          [power] [int] NULL,
                                          [price] [int] NULL,
                                          [seller] [nvarchar](128) NULL );
 
DROP TABLE IF EXISTS dbo.Sauto_for_anchoring78;
CREATE TABLE dbo.Sauto_for_anchoring78 ([make] [nvarchar](20) NULL,
                                        [model] [nvarchar](20) NULL,
                                        [make_model] [nvarchar](40) NULL,
                                        [makeyear] [int] NULL,
                                        [kilometer] [int] NULL,
                                        [power] [int] NULL,
                                        [price] [int] NULL  );

INSERT INTO dbo.Sauto_for_anchors_allMM
SELECT [Make],[Model],[Make]+[Model] AS [make_model],[Makeyear],[Tachometer],[Power],[Price],[SellerName]
FROM [CZPHAAASQL16\DM].[DataMining2].[dbo].[DmXmls] WHERE ServerId=1  AND
     [DateIn] BETWEEN '2017-01-01' AND '2018-03-31';

INSERT dbo.Sauto_for_anchoring78 EXEC
sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

cars = cars[(cars.seller!="AAA AUTO")&(cars.seller!="Mototechna")&(cars.seller!="AUTO DISKONT")]
del cars ["seller"]
del cars ["m_m_comb_dm"]
cars = cars.rename(columns={"m_m_comb_navi":"make_model_trans"})
cars["make_model_trans"].fillna("0",inplace=True)
missing_trans = cars[cars.make_model_trans=="0"] 
cars.loc[cars.make_model_trans=="0","make_model_trans"] = cars["make_model"]
del cars["make_model"]
cars = cars.rename(columns={"make_model_trans":"make_model"})
cars.dropna( inplace=True)
cars["make_model"] = cars["make_model"].apply(lambda mm: mm.lower()) 
cols_to_int = ["makeyear","kilometer","power","price"]
ctilen = len(cols_to_int)
for i in range(0, ctilen):
    cars[cols_to_int[i]] = cars[cols_to_int[i]].apply(lambda ct: int(ct))
cols_to_keep = ["make","model","make_model","makeyear","kilometer","power","price"]  
cars = cars[cols_to_keep]
cars.dropna(inplace=True)
cars = cars[(cars.makeyear>0)&(cars.kilometer>0)&(cars.power>0)&(cars.price>0)]
OutputDataSet = cars
',
@input_data_1 = N'
SELECT * FROM dbo.Sauto_for_anchors_allMM ss JOIN dbo.trans_make_model tm 
         ON ss.[make_model] = tm.[m_m_comb_dm] 
',
@input_data_1_name = N'cars',
@output_data_1_name = N'cars' 

END


In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_brand_mmcode_calculating78] 
        Script Date: 6/29/2018 3:32:45 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_brand_mmcode_calculating78] 
AS
BEGIN
DROP TABLE IF EXISTS dbo.brand_DB_mmcode78
CREATE TABLE dbo.brand_DB_mmcode78 ([make_model] [nvarchar](40) NULL,
                                    [counts] [int] NULL, 
                                    [mmcode] [int] NULL );

INSERT dbo.brand_DB_mmcode78 EXEC
sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

allbrands = cars["make_model"].unique()
blen = len(allbrands)
brand_count = [0]*blen
for i in range(0,blen):
    brand_count[i] = cars[cars.make_model==allbrands[i]]["make_model"].count()
brandDF = pd.DataFrame()
brandDF["make_model"] = allbrands    
brandDF["counts"] = brand_count   
brandDF = brandDF.sort_values("counts",ascending=False)  
brandDF["mmcode"] = np.arange(blen)
brands = brandDF["make_model"].unique()
brandDF.dropna( inplace=True)
OutputDataSet = brandDF
',
@input_data_1 = N'
SELECT * FROM dbo.Sauto_for_anchoring78
',
@input_data_1_name = N'cars',
@output_data_1_name = N'brandDF' 

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_anchors_calculating78]    Script Date: 6/29/2018 3:33:31 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_anchors_calculating78] 
AS
BEGIN
DROP TABLE IF EXISTS dbo.Sauto_all20178_DB_anchors
CREATE TABLE dbo.Sauto_all20178_DB_anchors ([make_model] [nvarchar](40) NULL,
                                            [mmcode] [int] NULL,
                                            [age_years] [int] NULL,
                                            [km_range] [int] NULL,
                                            [power] [int] NULL,
                                            [anchor] [int] NULL );

INSERT dbo.Sauto_all20178_DB_anchors
EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

def to_2000(yy):
    if (yy<2000):
        return 2000
    return yy
def over_2017(yy):
    if (yy>2017):
        return 2017
    return yy
cars["makeyear"] = cars["makeyear"].apply(to_2000)
cars["makeyear"] = cars["makeyear"].apply(over_2017)
cars["age"] = cars["makeyear"].apply(lambda my: 2018-my)
calen = len(cars)
def find_range(myp):
    if (myp<=10):
        return 0
    if ((myp>10)&(myp<=30)):
        return 1
    if ((myp>30)&(myp<=50)):
        return 2
    if ((myp>50)&(myp<=70)):
        return 3
    if ((myp>70)&(myp<=90)):
        return 4
    if ((myp>90)&(myp<=110)):
        return 5
    if ((myp>110)&(myp<=130)):
        return 6
    if ((myp>130)&(myp<=150)):
        return 7
    if ((myp>150)&(myp<=170)):
        return 8
    if ((myp>170)&(myp<=190)):
        return 9
    if ((myp>190)&(myp<=220)):
        return 10
    if ((myp>220)&(myp<=250)):
        return 11
    if ((myp>250)&(myp<=300)):
        return 12
    return 13

kilometer_range_count = pd.DataFrame()
ranglen = 14
cars["kilometer"] = cars["kilometer"].apply(lambda km: float("%.3f" %(km/1000)))
cars["km_range"] = cars["kilometer"].apply(find_range)
cars["km_range"] = cars["km_range"].apply(lambda pr:int(pr))
cars.sort_values("km_range",ascending=True, inplace=True)
rangcount = [0]*ranglen
rangprice = [0]*ranglen
for i in range(0, ranglen):
    rangcount[i] = cars[cars.km_range==i]["km_range"].count()
    rangprice[i] = cars[cars.km_range==i]["price"].median()
kilometer_range_count["anchoring_count"] = rangcount 
kilometer_range_count["price"] = rangprice
kilometer_range_count["price"] = kilometer_range_count["price"].apply(lambda mk: int(mk)) 
allbrands = cars["make_model"].unique()
blen = len(allbrands)
brand_count = [0]*blen
for i in range(0,blen):
    brand_count[i] = cars[cars.make_model==allbrands[i]]["make_model"].count()
brandDF = pd.DataFrame()
brandDF["make_model"] = allbrands    
brandDF["counts"] = brand_count   
brandDF = brandDF.sort_values("counts",ascending=False)  
brandDF["mmcode"] = np.arange(blen)
brands = brandDF["make_model"].unique()
brandDF.dropna( inplace=True)
cars = pd.merge(cars, brandDF, on="make_model", how="outer")
cars.sort_values(["mmcode","makeyear","km_range","power"], ascending=[True,False,True,True], inplace=True )
cars_meds = cars.groupby(["mmcode","make_model","age","km_range","power"])["price"].median()
cars_medians = cars_meds.to_frame()
cars_medians = cars_medians.reset_index(level=["mmcode","make_model","age","km_range","power"])
cars_medians = cars_medians.rename(columns={"age":"age_years","price":"anchor"})
cars_medians["anchor"] = cars_medians["anchor"].apply(lambda pp: int(pp))
coltk = ["make_model","mmcode","age_years","km_range", "power","anchor"]
cars_medians = cars_medians[coltk]

OutputDataSet = cars_medians
',
@input_data_1 = N'
SELECT * FROM dbo.Sauto_for_anchoring78
',
@input_data_1_name = N'cars',
@output_data_1_name = N'cars_medians' 

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_map_anchors_calculating78]    Script Date: 7/2/2018 3:48:20 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_map_anchors_calculating78] 
AS
BEGIN
DROP TABLE IF EXISTS dbo.Sauto_all20178_DB_map_anchors
CREATE TABLE dbo.Sauto_all20178_DB_map_anchors ([make_model] [nvarchar](40) NULL,
                                                [mmcode] [int] NULL,
                                                [age_years] [int] NULL,
                                                [power] [int] NULL,
                                                [map_anchor] [int] NULL );

INSERT dbo.Sauto_all20178_DB_map_anchors
EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

def to_2000(yy):
    if (yy<2000):
        return 2000
    return yy
def over_2017(yy):
    if (yy>2017):
        return 2017
    return yy
cars["makeyear"] = cars["makeyear"].apply(to_2000)
cars["makeyear"] = cars["makeyear"].apply(over_2017)
cars["age"] = cars["makeyear"].apply(lambda my: 2018-my)
calen = len(cars)
allbrands = cars["make_model"].unique()
blen = len(allbrands)
brand_count = [0]*blen
for i in range(0,blen):
    brand_count[i] = cars[cars.make_model==allbrands[i]]["make_model"].count()
brandDF = pd.DataFrame()
brandDF["make_model"] = allbrands    
brandDF["counts"] = brand_count   
brandDF = brandDF.sort_values("counts",ascending=False)  
brandDF["mmcode"] = np.arange(blen)
brands = brandDF["make_model"].unique()
brandDF.dropna( inplace=True)
cars = pd.merge(cars, brandDF, on="make_model", how="outer")
cars_meds = cars.groupby(["mmcode","make_model","age","power"])["price"].median()
cars_medians = cars_meds.to_frame()
cars_medians = cars_medians.reset_index(level=["mmcode","make_model","age","power"])
cars_medians = cars_medians.rename(columns={"age":"age_years","price":"anchor"})
cars_medians["anchor"] = cars_medians["anchor"].apply(lambda pp: int(pp))
coltk = ["make_model","mmcode","age_years","power","anchor"]
cars_medians = cars_medians[coltk]

OutputDataSet = cars_medians
',
@input_data_1 = N'
SELECT * FROM dbo.Sauto_for_anchoring78
',
@input_data_1_name = N'cars',
@output_data_1_name = N'cars_medians' 

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_mm_years_anchors_calculating78] 
        Script Date: 6/29/2018 3:34:24 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_mm_years_anchors_calculating78] 
AS
BEGIN

DROP TABLE IF EXISTS dbo.Sauto_all20178_DB_mm_years_anchors
CREATE TABLE dbo.Sauto_all20178_DB_mm_years_anchors ([make_model] [nvarchar](40) NULL,
                                                     [mmcode] [int] NULL,
                                                     [make_year] [int] NULL,
                                                     [anchor] [int] NULL );

INSERT dbo.Sauto_all20178_DB_mm_years_anchors
EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

def to_2000(yy):
    if (yy<2000):
        return 2000
    return yy
def over_2017(yy):
    if (yy>2017):
        return 2017
    return yy
cars["makeyear"] = cars["makeyear"].apply(to_2000)
cars["makeyear"] = cars["makeyear"].apply(over_2017)
cars.sort_values(["mmcode","makeyear"], ascending=[True,False], inplace=True )
cars_years_meds = cars.groupby(["mmcode","make_model","makeyear"])["price"].median()
cars_years_medians = cars_years_meds.to_frame()
cars_years_medians = cars_years_medians.reset_index(level=["mmcode","make_model","makeyear"])
cars_years_medians = cars_years_medians.rename(columns={"makeyear":"make_year","price":"anchor"})
coltk = ["make_model","mmcode","make_year","anchor"]
cars_years_medians["anchor"] = cars_years_medians["anchor"].apply(lambda cc:int(cc))
cars_years_medians = cars_years_medians[coltk]
OutputDataSet = cars_years_medians
',
@input_data_1 = N'
SELECT ss.make_model, tm.mmcode, ss.makeyear, ss.price FROM dbo.Sauto_for_anchoring78 ss 
JOIN dbo.brand_DB_mmcode78 tm ON ss.[make_model] = tm.[make_model] ;
',
@input_data_1_name = N'cars',
@output_data_1_name = N'cars_years_medians' 

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[Sauto_make_model_anchors_calculating78]
        Script Date: 6/29/2018 3:39:23 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[Sauto_make_model_anchors_calculating78] 
AS
BEGIN
DROP TABLE IF EXISTS dbo.Sauto_all20178_DB_make_model_anchors
CREATE TABLE dbo.Sauto_all20178_DB_make_model_anchors ([make_model] [nvarchar](40) NULL,
                                                       [mmcode] [int] NULL,
                                                       [anchor] [int] NULL );


INSERT dbo.Sauto_all20178_DB_make_model_anchors
EXEC sp_execute_external_script
@language = N'Python',
@script = N'
import pandas as pd
from ipykernel import kernelapp as app
import pypyodbc as podbc
import numpy as np

cars.sort_values(["mmcode"], ascending=True, inplace=True )
cars_meds = cars.groupby(["mmcode","make_model"])["price"].median()
cars_medians = cars_meds.to_frame()
cars_medians = cars_medians.reset_index(level=["mmcode","make_model"])
cars_medians = cars_medians.rename(columns={"price":"anchor"})
coltk = ["make_model","mmcode","anchor"]
cars_medians["anchor"] = cars_medians["anchor"].apply(lambda cc:int(cc))
cars_medians = cars_medians[coltk]
OutputDataSet = cars_medians
',
@input_data_1 = N'
SELECT ss.make_model, tm.mmcode, ss.price FROM dbo.Sauto_for_anchoring78 ss 
JOIN dbo.brand_DB_mmcode78 tm ON ss.[make_model] = tm.[make_model] ;
',
@input_data_1_name = N'cars',
@output_data_1_name = N'cars_medians' 

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[calculations_121_limit68_DB_RAFO_models]  
        Script Date: 7/2/2018 4:30:35 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[calculations_121_limit68_DB_RAFO_models]
AS
BEGIN

DROP TABLE IF EXISTS dbo.RAFO_limit68_DB_121models

CREATE TABLE dbo.RAFO_limit68_DB_121models
(   car_segment INT,
    make_model VARCHAR(40),
    rafo_model VARBINARY(MAX)
) 
DECLARE @segcount INT = 121
DECLARE @segment INT = 1
DECLARE @trained_model VARBINARY(MAX)

WHILE (@segment < @segcount)  
BEGIN 
DECLARE @make_model VARCHAR(40)=(SELECT make_model FROM TSorder_DB_codes68
                                                   WHERE tscode = @segment);
EXEC [dbo].[calculation_RAFO_limitDB_uni_model] 
@make_model=@make_model, @trained_model = @trained_model OUTPUT

INSERT INTO RAFO_limit68_DB_121models (car_segment,make_model,rafo_model) 
VALUES(@segment,@make_model, @trained_model);
SET @segment = @segment + 1
END

DECLARE @trained_model0 VARBINARY(MAX)
EXEC dbo.calculation_RAFO_limitDB_121st_model @trained_model= @trained_model0 OUTPUT

INSERT INTO RAFO_limit68_DB_121models (car_segment,make_model,rafo_model) 
VALUES(121,'outTop120', @trained_model0);

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[calculation_RAFO_limitDB_uni_model] 
        Script Date: 7/2/2018 4:32:00 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[calculation_RAFO_limitDB_uni_model]
                                  (@make_model VARCHAR(40), @trained_model varbinary(max) OUTPUT)
AS
BEGIN
  EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
    
import pandas as pd
import numpy as np
import pickle
from ipykernel import kernelapp as app
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

prediction_var = ["body_cd","fuel_cd","revkm","revage","power","sizeccm","automat","drive","mec_con",
                  "ext_con","isfirstowner","isservicebook","isvat","autoaircondition","airconditioned",
                  "isxenon","isled","isnavigation","iscruisecontrol","isinleather","airbags","anchor"]
mycars = cars[cars.make_model==make_model]
rafo = RandomForestRegressor(n_estimators=33, max_depth=33, max_features=16, random_state=42 )
X = mycars[prediction_var]
y = mycars["price"]
X_tr, X_test, y_tr, y_test = train_test_split(X,y,test_size=0.10,random_state=42)
rafo.fit(X_tr,y_tr) 
trained_model = pickle.dumps(rafo)
',

@input_data_1 = N'
SELECT * FROM dbo.sold2016_8_DB_anchored_cars
',
@input_data_1_name = N'cars',
@params = N'@make_model VARCHAR(40), @trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT,
@make_model = @make_model 
;

END

In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[calculation_RAFO_limitDB_121st_model] 
        Script Date: 7/2/2018 4:32:37 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
ALTER PROCEDURE [dbo].[calculation_RAFO_limitDB_121st_model]( @trained_model varbinary(max) OUTPUT)
AS
BEGIN

  EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
    
import pandas as pd
import numpy as np
import pickle
from ipykernel import kernelapp as app
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

cars = cars[cars.tscode>120]
prediction_var = ["make_cd","model_cd","body_cd","fuel_cd","revkm","revage","power","sizeccm","automat",
                  "drive","mec_con","ext_con","isfirstowner","isservicebook","isvat","autoaircondition",
                  "airconditioned","isxenon","isled","isnavigation","iscruisecontrol","isinleather",
                  "airbags","anchor"]
rafo = RandomForestRegressor(n_estimators=33, max_depth=33, max_features=18, random_state=42 )
X = cars[prediction_var]
y = cars["price"]
X_tr, X_test, y_tr, y_test = train_test_split(X,y,test_size=0.10,random_state=42)
rafo.fit(X_tr,y_tr) 
trained_model = pickle.dumps(rafo)
',

@input_data_1 = N'
SELECT * FROM dbo.sold2016_8_DB_anchored_cars
',
@input_data_1_name = N'cars',
@params = N' @trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT
;
END



In [ ]:
USE [Pricing]
GO
/****** Object:  StoredProcedure [dbo].[all_on_click]    Script Date: 7/2/2018 4:25:20 PM ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER PROCEDURE [dbo].[all_on_click] 
AS
BEGIN
EXEC dbo.Sauto_preprocessing78 ;
EXEC dbo.Sauto_brand_mmcode_calculating78 ;
EXEC dbo.Sauto_anchors_calculating78 ;
EXEC dbo.Sauto_map_anchors_calculating78 ;
EXEC dbo.Sauto_mm_years_anchors_calculating78;
EXEC dbo.Sauto_make_model_anchors_calculating78;
EXEC dbo.sold20168_DB_TS_cars_processing;      /*** calls dbo.sold20168_TS_cars_MMcoding ***/
EXEC dbo.sold20168_DB_TS_cars_anchoring;       /*** calls dbo.sold20168_TS_cars_ts_ordering ***/
EXEC dbo.calculations_121_limit68_DB_RAFO_models; /*** calls dbo.calculation_RAFO_limitDB_uni_model ***/
                                              /*** calls dbo.calculation_RAFO_limitDB__121st_model ***/
EXEC dbo.bought2018_Test_cars_processing46 ;
EXEC dbo.bought2018_Test_cars_anchoring ;

END